In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d uciml/breast-cancer-wisconsin-data
!unzip \*.zip && rm *.zip

  0% 0.00/48.6k [00:00<?, ?B/s]
100% 48.6k/48.6k [00:00<00:00, 54.7MB/s]
Archive:  breast-cancer-wisconsin-data.zip
  inflating: data.csv                


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Step 1: Load Data
data = pd.read_csv('/content/data.csv')

# Assuming 'diagnosis' is the target variable
X = data[['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
          'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean']]
y = data['diagnosis']  # Target variable

# Step 2: Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Define the fuzzy membership function
def fuzzy_membership(distance, threshold):
    if distance <= threshold:
        return 1.0
    else:
        return np.exp(-distance / threshold)

# Step 4: Apply FMKNN algorithm
def fmknn(X_train, y_train, X_test, k, threshold):
    y_pred = []
    for x_test in X_test:
        distances = np.linalg.norm(X_train - x_test, axis=1)
        indices = np.argsort(distances)[:k]
        fuzzy_memberships = [fuzzy_membership(distances[i], threshold) for i in indices]
        labels = [y_train[i] for i in indices]
        counts = {}
        for label, membership in zip(labels, fuzzy_memberships):
            if label in counts:
                counts[label] += membership
            else:
                counts[label] = membership
        pred_label = max(counts, key=counts.get)
        y_pred.append(pred_label)
    return y_pred

# Step 5: Define parameters
k = 5  # Number of neighbors
threshold = 2.0  # Threshold for fuzzy membership

# Step 6: Apply FMKNN algorithm
y_pred = fmknn(X_train.to_numpy(), y_train.to_numpy(), X_test.to_numpy(), k, threshold)

# Step 7: Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8859649122807017
Precision: 0.8855203888098625
Recall: 0.8859649122807017
F1 Score: 0.8850837105226715
Classification Report:
               precision    recall  f1-score   support

           B       0.89      0.93      0.91        71
           M       0.88      0.81      0.84        43

    accuracy                           0.89       114
   macro avg       0.88      0.87      0.88       114
weighted avg       0.89      0.89      0.89       114

